In [1]:
import requests
import pandas as pd
import json
import csv
import numpy as np
import seaborn as sns

from scipy import stats
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=200000"

    
response = requests.get(url)

In [3]:
response.status_code

200

In [4]:
print(type(response))

<class 'requests.models.Response'>


In [5]:
temp = response.json
json_obj = json.dumps(temp())

In [6]:
with open("response.json","w") as outfile:
    outfile.write(json_obj)

In [7]:
with open("response.json") as json_file:
    data = json.load(json_file)
    
raw_data = data["result"]["records"]

data_file = open("data_file.csv", "w")
csv_writer = csv.writer(data_file)

count = 0

for data in raw_data:
    if count == 0:
        header = data.keys()
        csv_writer.writerow(header)
        count+=1
        
    csv_writer.writerow(data.values())
    
data_file.close()    

In [69]:
df = pd.read_csv("data_file.csv")
df.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block
0,WOODLANDS,5 ROOM,Model A,134.0,MARSILING RD,350000.0,2017-03,77 years 01 month,1995,04 TO 06,1,139
1,WOODLANDS,5 ROOM,Standard,120.0,MARSILING DR,355000.0,2017-03,57 years 10 months,1976,16 TO 18,2,9
2,WOODLANDS,5 ROOM,Standard,126.0,MARSILING DR,370000.0,2017-03,58 years,1976,01 TO 03,3,10
3,WOODLANDS,5 ROOM,Improved,120.0,WOODLANDS DR 60,370000.0,2017-03,79 years 07 months,1997,04 TO 06,4,786D
4,WOODLANDS,5 ROOM,Premium Apartment,110.0,WOODLANDS DR 75,370000.0,2017-03,86 years,2004,10 TO 12,5,688B


In [70]:
df.shape

(105789, 12)

In [71]:
df["town"].value_counts()

SENGKANG           8901
PUNGGOL            7668
WOODLANDS          7405
JURONG WEST        7248
TAMPINES           7150
YISHUN             7122
BEDOK              5754
HOUGANG            5112
CHOA CHU KANG      4637
ANG MO KIO         4546
BUKIT PANJANG      4207
BUKIT MERAH        4105
BUKIT BATOK        3554
TOA PAYOH          3409
PASIR RIS          3258
QUEENSTOWN         3037
KALLANG/WHAMPOA    3000
GEYLANG            2589
SEMBAWANG          2492
CLEMENTI           2400
JURONG EAST        2270
BISHAN             2077
SERANGOON          2043
CENTRAL AREA        864
MARINE PARADE       651
BUKIT TIMAH         290
Name: town, dtype: int64

In [72]:
df["flat_type"].value_counts()

4 ROOM              44010
5 ROOM              26974
3 ROOM              24667
EXECUTIVE            8456
2 ROOM               1582
MULTI-GENERATION       54
1 ROOM                 46
Name: flat_type, dtype: int64

In [73]:
df['flat_model'].value_counts()

Model A                   34840
Improved                  26520
New Generation            13671
Premium Apartment         12006
Apartment                  4257
Simplified                 4151
Maisonette                 3204
Standard                   2963
DBSS                       2023
Model A2                   1243
Adjoined flat               202
Model A-Maisonette          191
Type S1                     178
Type S2                     107
Premium Apartment Loft       70
Terrace                      68
Multi Generation             54
Improved-Maisonette          18
2-room                       12
Premium Maisonette           11
Name: flat_model, dtype: int64

In [74]:
df["storey_range"].value_counts()

04 TO 06    24475
07 TO 09    22088
10 TO 12    19708
01 TO 03    18881
13 TO 15    10101
16 TO 18     4709
19 TO 21     2016
22 TO 24     1511
25 TO 27      851
28 TO 30      527
31 TO 33      265
34 TO 36      245
37 TO 39      236
40 TO 42      115
43 TO 45       29
46 TO 48       25
49 TO 51        7
Name: storey_range, dtype: int64

In [75]:
df["flat_type"].str.split()

0           [5, ROOM]
1           [5, ROOM]
2           [5, ROOM]
3           [5, ROOM]
4           [5, ROOM]
             ...     
105784      [5, ROOM]
105785      [5, ROOM]
105786      [5, ROOM]
105787    [EXECUTIVE]
105788    [EXECUTIVE]
Name: flat_type, Length: 105789, dtype: object

In [76]:
df.replace(to_replace= ["EXECUTIVE","MULTI-GENERATION"], value=["6 ROOM", "7 ROOM"], inplace= True)

In [77]:
df["flat_type"].value_counts()

4 ROOM    44010
5 ROOM    26974
3 ROOM    24667
6 ROOM     8456
2 ROOM     1582
7 ROOM       54
1 ROOM       46
Name: flat_type, dtype: int64

In [78]:
df["flat_type"].str.split().str[0]

0         5
1         5
2         5
3         5
4         5
         ..
105784    5
105785    5
105786    5
105787    6
105788    6
Name: flat_type, Length: 105789, dtype: object

In [79]:
df["Total_rooms"] = df["flat_type"].str.split().str[0]

In [80]:
df["Min_storeys"] = df["storey_range"].str.split().str[0]
df["Max_storeys"] = df["storey_range"].str.split().str[2]

In [81]:
df["remaining_lease"].str.split().str[0]

0         77
1         57
2         58
3         79
4         86
          ..
105784    94
105785    65
105786    65
105787    65
105788    65
Name: remaining_lease, Length: 105789, dtype: object

In [82]:
df["remaining_years_of_lease"] = df["remaining_lease"].str.split().str[0]

In [83]:
df.isnull().sum()

town                        0
flat_type                   0
flat_model                  0
floor_area_sqm              0
street_name                 0
resale_price                0
month                       0
remaining_lease             0
lease_commence_date         0
storey_range                0
_id                         0
block                       0
Total_rooms                 0
Min_storeys                 0
Max_storeys                 0
remaining_years_of_lease    0
dtype: int64

In [84]:
df.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,Total_rooms,Min_storeys,Max_storeys,remaining_years_of_lease
0,WOODLANDS,5 ROOM,Model A,134.0,MARSILING RD,350000.0,2017-03,77 years 01 month,1995,04 TO 06,1,139,5,04,06,77
1,WOODLANDS,5 ROOM,Standard,120.0,MARSILING DR,355000.0,2017-03,57 years 10 months,1976,16 TO 18,2,9,5,16,18,57
2,WOODLANDS,5 ROOM,Standard,126.0,MARSILING DR,370000.0,2017-03,58 years,1976,01 TO 03,3,10,5,01,03,58
3,WOODLANDS,5 ROOM,Improved,120.0,WOODLANDS DR 60,370000.0,2017-03,79 years 07 months,1997,04 TO 06,4,786D,5,04,06,79
4,WOODLANDS,5 ROOM,Premium Apartment,110.0,WOODLANDS DR 75,370000.0,2017-03,86 years,2004,10 TO 12,5,688B,5,10,12,86


In [85]:
town_lst = np.sort(df.town.unique()).tolist()
len(town_lst)

26

In [86]:
df["resale_price"].median()

425000.0

In [87]:
df.groupby("town")["resale_price"].median()

town
ANG MO KIO         350000.0
BEDOK              380000.0
BISHAN             625000.0
BUKIT BATOK        368000.0
BUKIT MERAH        590000.0
BUKIT PANJANG      430000.0
BUKIT TIMAH        719000.0
CENTRAL AREA       512500.0
CHOA CHU KANG      400000.0
CLEMENTI           425000.0
GEYLANG            398000.0
HOUGANG            420000.0
JURONG EAST        405000.0
JURONG WEST        398000.0
KALLANG/WHAMPOA    478000.0
MARINE PARADE      465000.0
PASIR RIS          492500.0
PUNGGOL            460000.0
QUEENSTOWN         620000.0
SEMBAWANG          382000.0
SENGKANG           440000.0
SERANGOON          473000.0
TAMPINES           465000.0
TOA PAYOH          420000.0
WOODLANDS          380000.0
YISHUN             358000.0
Name: resale_price, dtype: float64

In [88]:
ser = df["resale_price"].median() - df.groupby("town")["resale_price"].median()


town_prem = pd.DataFrame({"town": ser.index, "town_premium": ser.values})
town_prem.to_csv("town_prem")

In [89]:
town_prem

,town,town_premium
0,ANG MO KIO,75000.0
1,BEDOK,45000.0
2,BISHAN,-200000.0
3,BUKIT BATOK,57000.0
4,BUKIT MERAH,-165000.0
5,BUKIT PANJANG,-5000.0
6,BUKIT TIMAH,-294000.0
7,CENTRAL AREA,-87500.0
8,CHOA CHU KANG,25000.0
9,CLEMENTI,0.0


In [90]:
df = pd.merge(df, town_prem, left_on = "town", right_on = "town")
df

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,Total_rooms,Min_storeys,Max_storeys,remaining_years_of_lease,town_premium
0,WOODLANDS,5 ROOM,Model A,134.0,MARSILING RD,350000.0,2017-03,77 years 01 month,1995,04 TO 06,1,139,5,04,06,77,45000.0
1,WOODLANDS,5 ROOM,Standard,120.0,MARSILING DR,355000.0,2017-03,57 years 10 months,1976,16 TO 18,2,9,5,16,18,57,45000.0
2,WOODLANDS,5 ROOM,Standard,126.0,MARSILING DR,370000.0,2017-03,58 years,1976,01 TO 03,3,10,5,01,03,58,45000.0
3,WOODLANDS,5 ROOM,Improved,120.0,WOODLANDS DR 60,370000.0,2017-03,79 years 07 months,1997,04 TO 06,4,786D,5,04,06,79,45000.0
4,WOODLANDS,5 ROOM,Premium Apartment,110.0,WOODLANDS DR 75,370000.0,2017-03,86 years,2004,10 TO 12,5,688B,5,10,12,86,45000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105784,TOA PAYOH,4 ROOM,Improved,87.0,TOA PAYOH NTH,400000.0,2021-10,50 years 11 months,1973,10 TO 12,105723,200,4,10,12,50,5000.0
105785,TOA PAYOH,4 ROOM,Improved,81.0,TOA PAYOH NTH,390000.0,2021-10,50 years 10 months,1973,07 TO 09,105724,208,4,07,09,50,5000.0
105786,TOA PAYOH,5 ROOM,Improved,120.0,JOO SENG RD,570000.0,2021-10,65 years 10 months,1988,01 TO 03,105725,19,5,01,03,65,5000.0
105787,TOA PAYOH,5 ROOM,Improved,120.0,LOR 4 TOA PAYOH,903888.0,2021-10,77 years 01 month,1999,22 TO 24,105726,60,5,22,24,77,5000.0


In [91]:
df.groupby("flat_model")["resale_price"].median()

flat_model
2-room                     259500.0
Adjoined flat              680000.0
Apartment                  610000.0
DBSS                       748000.0
Improved                   452000.0
Improved-Maisonette        687500.0
Maisonette                 668000.0
Model A                    410000.0
Model A-Maisonette         725000.0
Model A2                   346000.0
Multi Generation           799444.0
New Generation             320000.0
Premium Apartment          468000.0
Premium Apartment Loft     900000.0
Premium Maisonette         720000.0
Simplified                 330000.0
Standard                   315000.0
Terrace                    825000.0
Type S1                    948444.0
Type S2                   1050000.0
Name: resale_price, dtype: float64

In [92]:
ser_2 = df["resale_price"].median() - df.groupby("flat_model")["resale_price"].median()
ser_2

flat_model
2-room                    165500.0
Adjoined flat            -255000.0
Apartment                -185000.0
DBSS                     -323000.0
Improved                  -27000.0
Improved-Maisonette      -262500.0
Maisonette               -243000.0
Model A                    15000.0
Model A-Maisonette       -300000.0
Model A2                   79000.0
Multi Generation         -374444.0
New Generation            105000.0
Premium Apartment         -43000.0
Premium Apartment Loft   -475000.0
Premium Maisonette       -295000.0
Simplified                 95000.0
Standard                  110000.0
Terrace                  -400000.0
Type S1                  -523444.0
Type S2                  -625000.0
Name: resale_price, dtype: float64

In [93]:
ser_2 = df["resale_price"].median() - df.groupby("flat_model")["resale_price"].median()


flat_prem = pd.DataFrame({"flat_model": ser_2.index, "flat_premium": ser_2.values})
flat_prem.to_csv("flat_prem")

In [94]:
flat_prem

,flat_model,flat_premium
0,2-room,165500.0
1,Adjoined flat,-255000.0
2,Apartment,-185000.0
3,DBSS,-323000.0
4,Improved,-27000.0
5,Improved-Maisonette,-262500.0
6,Maisonette,-243000.0
7,Model A,15000.0
8,Model A-Maisonette,-300000.0
9,Model A2,79000.0


In [95]:
df = pd.merge(df, flat_prem, left_on = "flat_model", right_on = "flat_model")
df.head(20)

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,Total_rooms,Min_storeys,Max_storeys,remaining_years_of_lease,town_premium,flat_premium
0,WOODLANDS,5 ROOM,Model A,134.0,MARSILING RD,350000.0,2017-03,77 years 01 month,1995,04 TO 06,1,139,5,04,06,77,45000.0,15000.0
1,WOODLANDS,5 ROOM,Model A,132.0,WOODLANDS ST 31,420000.0,2017-03,74 years 11 months,1993,01 TO 03,30,304,5,01,03,74,45000.0,15000.0
2,WOODLANDS,3 ROOM,Model A,74.0,MARSILING CRES,260000.0,2017-04,64 years 10 months,1983,01 TO 03,1758,212,3,01,03,64,45000.0,15000.0
3,WOODLANDS,3 ROOM,Model A,73.0,WOODLANDS ST 13,280000.0,2017-04,66 years 11 months,1985,04 TO 06,1762,152,3,04,06,66,45000.0,15000.0
4,WOODLANDS,3 ROOM,Model A,73.0,WOODLANDS ST 31,289000.0,2017-04,75 years 05 months,1993,01 TO 03,1766,314,3,01,03,75,45000.0,15000.0
5,WOODLANDS,3 ROOM,Model A,83.0,WOODLANDS ST 31,293000.0,2017-04,75 years 08 months,1993,10 TO 12,1767,318,3,10,12,75,45000.0,15000.0
6,WOODLANDS,3 ROOM,Model A,67.0,MARSILING LANE,340000.0,2017-04,95 years 02 months,2013,25 TO 27,1768,12A,3,25,27,95,45000.0,15000.0
7,WOODLANDS,4 ROOM,Model A,90.0,MARSILING DR,300000.0,2017-04,84 years,2002,10 TO 12,1769,5A,4,10,12,84,45000.0,15000.0
8,WOODLANDS,4 ROOM,Model A,104.0,MARSILING CRES,318000.0,2017-04,64 years 06 months,1982,01 TO 03,1776,213,4,01,03,64,45000.0,15000.0
9,WOODLANDS,4 ROOM,Model A,101.0,WOODLANDS CIRCLE,325000.0,2017-04,79 years 09 months,1998,04 TO 06,1782,751,4,04,06,79,45000.0,15000.0


In [96]:
df["Registration_year"] = df["month"].str.split("-").str[0]

In [97]:
y = df["resale_price"]

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105789 entries, 0 to 105788
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   town                      105789 non-null  object 
 1   flat_type                 105789 non-null  object 
 2   flat_model                105789 non-null  object 
 3   floor_area_sqm            105789 non-null  float64
 4   street_name               105789 non-null  object 
 5   resale_price              105789 non-null  float64
 6   month                     105789 non-null  object 
 7   remaining_lease           105789 non-null  object 
 8   lease_commence_date       105789 non-null  int64  
 9   storey_range              105789 non-null  object 
 10  _id                       105789 non-null  int64  
 11  block                     105789 non-null  object 
 12  Total_rooms               105789 non-null  object 
 13  Min_storeys               105789 non-null  o

In [99]:
df.drop("resale_price", axis=1, inplace=True)

In [100]:
to_num_cols = ["Total_rooms", "Min_storeys", "Max_storeys", "remaining_years_of_lease", "Registration_year"]
df[to_num_cols] = df[to_num_cols].apply(pd.to_numeric)

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105789 entries, 0 to 105788
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   town                      105789 non-null  object 
 1   flat_type                 105789 non-null  object 
 2   flat_model                105789 non-null  object 
 3   floor_area_sqm            105789 non-null  float64
 4   street_name               105789 non-null  object 
 5   month                     105789 non-null  object 
 6   remaining_lease           105789 non-null  object 
 7   lease_commence_date       105789 non-null  int64  
 8   storey_range              105789 non-null  object 
 9   _id                       105789 non-null  int64  
 10  block                     105789 non-null  object 
 11  Total_rooms               105789 non-null  int64  
 12  Min_storeys               105789 non-null  int64  
 13  Max_storeys               105789 non-null  i

In [ ]:
all_data = df.copy()
all_data["resale_price"] =  y

corrmat = all_data.corr(method = "spearman")
plt.figure(figsize=(15,15))
#plot heat map
g=sns.heatmap(corrmat,annot=True)

In [102]:
df.drop("Min_storeys", axis=1, inplace= True)

In [ ]:
all_data.describe()

In [65]:
col_lst = df.columns.tolist()
col_lst

['floor_area_sqm',
 'lease_commence_date',
 'Total_rooms',
 'Max_storeys',
 'remaining_years_of_lease',
 'town_premium',
 'flat_premium',
 'Registration_year']

In [ ]:
from scipy.stats import skew

for fea in col_lst:
    plt.figure(figsize=(10,10))
    sns.set_style('whitegrid')
    sns.distplot(df[fea], kde=True, bins= 50)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.title(fea)
    plt.show()
    print("The skewness is ", skew(df[fea]))

In [ ]:
for fea in col_lst:
    plt.figure(figsize=(10,10))
    sns.boxplot(df[fea])
    plt.show()

In [103]:
Q1_fa =  df["floor_area_sqm"].quantile(0.25)
Q3_fa =  df["floor_area_sqm"].quantile(0.75)

iqr_fa = Q3_fa - Q1_fa

up_lim_fa = Q3_fa + 1.5*iqr_fa
low_lim_fa = Q1_fa - 1.5*iqr_fa


Q1_tr =  df["Total_rooms"].quantile(0.25)
Q3_tr =  df["Total_rooms"].quantile(0.75)

iqr_tr = Q3_tr - Q1_tr

up_lim_tr = Q3_tr + 1.5*iqr_tr
low_lim_tr = Q1_tr - 1.5*iqr_tr

Q1_ms =  df["Max_storeys"].quantile(0.25)
Q3_ms =  df["Max_storeys"].quantile(0.75)

iqr_ms = Q3_ms - Q1_ms

up_lim_ms = Q3_ms + 1.5*iqr_ms
low_lim_ms = Q1_ms - 1.5*iqr_ms

Q1_fp =  df["flat_premium"].quantile(0.25)
Q3_fp =  df["flat_premium"].quantile(0.75)

iqr_fp = Q3_fp - Q1_fp

up_lim_fp = Q3_fp + 1.5*iqr_fp
low_lim_fp = Q1_fp - 1.5*iqr_fp

c1 = df["floor_area_sqm"] > up_lim_fa
c2 = df["floor_area_sqm"] < low_lim_fa
c3 = df["Total_rooms"] > up_lim_tr
c4 = df["Total_rooms"] < low_lim_tr
c5 = df["Max_storeys"] > up_lim_ms
c6 = df["Max_storeys"] < low_lim_ms
c7 = df["flat_premium"] > up_lim_fp
c8 = df["flat_premium"] < low_lim_fp


In [104]:
df.loc[c1, "floor_area_sqm"] = up_lim_fa
df.loc[c2, "floor_area_sqm"] = low_lim_fa
df.loc[c3, "Total_rooms"]  = up_lim_tr
df.loc[c4, "Total_rooms"]  = low_lim_tr
df.loc[c5, "Max_storeys"]  = up_lim_ms
df.loc[c6, "Max_storeys"]  = low_lim_ms
df.loc[c7, "flat_premium"] = up_lim_fp
df.loc[c8, "flat_premium"] = low_lim_fp

In [ ]:
low_lim_fp

In [ ]:
for fea in col_lst:
    print(fea)
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    df[fea].hist()
    plt.subplot(1, 2, 2)
    stats.probplot(df[fea], dist="norm", plot=plt)
    plt.show()

In [105]:
town_prem = df[["town", "town_premium"]].drop_duplicates()
town_prem.to_csv("town_prem.csv", index=False)

In [106]:
dict(zip(town_prem["town"], town_prem["town_premium"]))

{'WOODLANDS': 45000.0,
 'YISHUN': 67000.0,
 'ANG MO KIO': 75000.0,
 'BEDOK': 45000.0,
 'BISHAN': -200000.0,
 'BUKIT BATOK': 57000.0,
 'PUNGGOL': -35000.0,
 'BUKIT MERAH': -165000.0,
 'BUKIT PANJANG': -5000.0,
 'BUKIT TIMAH': -294000.0,
 'CENTRAL AREA': -87500.0,
 'CHOA CHU KANG': 25000.0,
 'CLEMENTI': 0.0,
 'GEYLANG': 27000.0,
 'HOUGANG': 5000.0,
 'JURONG EAST': 20000.0,
 'JURONG WEST': 27000.0,
 'KALLANG/WHAMPOA': -53000.0,
 'PASIR RIS': -67500.0,
 'QUEENSTOWN': -195000.0,
 'SEMBAWANG': 43000.0,
 'SENGKANG': -15000.0,
 'SERANGOON': -48000.0,
 'TAMPINES': -40000.0,
 'TOA PAYOH': 5000.0,
 'MARINE PARADE': -40000.0}

In [107]:
flat_prem = df[["flat_model", "flat_premium"]].drop_duplicates()
flat_prem.to_csv("flat_prem.csv", index=False)

In [145]:
flat_prem

,flat_model,flat_premium
0,Model A,15000.0
34840,Standard,78000.0
37803,Improved,-27000.0
64323,Premium Apartment,-43000.0
76329,Apartment,-90000.0
80586,Maisonette,-90000.0
83790,New Generation,78000.0
97461,Simplified,78000.0
101612,Model A2,78000.0
102855,Adjoined flat,-90000.0


In [108]:
to_drop = ["town", "storey_range", "_id", "block", "remaining_lease", "flat_model", "flat_type", "street_name", "month"]
df.drop(to_drop, axis=1, inplace=True)

In [109]:
df.columns

Index(['floor_area_sqm', 'lease_commence_date', 'Total_rooms', 'Max_storeys',
       'remaining_years_of_lease', 'town_premium', 'flat_premium',
       'Registration_year'],
      dtype='object')

In [110]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=1)

In [111]:
X_train.to_csv("train_data.csv", index=False)

In [ ]:
pd.read_csv("train_data.csv")

In [112]:
y_train.to_csv("y_train.csv", index=False)

In [ ]:
y_train

In [ ]:
pd.read_csv("y_train.csv")

In [136]:
X_train.head(1)
            

,floor_area_sqm,lease_commence_date,Total_rooms,Max_storeys,remaining_years_of_lease,town_premium,flat_premium,Registration_year
68996,93.0,2017,4.0,12.0,95,-35000.0,-43000.0,2021


In [137]:
X_test.head(1)

,floor_area_sqm,lease_commence_date,Total_rooms,Max_storeys,remaining_years_of_lease,town_premium,flat_premium,Registration_year
82015,148.0,1992,6.0,9.0,74,5000.0,-90000.0,2017


In [139]:
X_tst_sc[0]

array([ 2.09527292, -0.22384279,  2.02984719, -0.08200504, -0.03113887,
        0.18981967, -1.80138779, -1.55238858])

In [147]:
model_rf.predict([[ 2.09527292, -0.22384279,  2.02984719, -0.08200504, -0.03113887,
        0.18981967, -1.80138779, -1.55238858]])

array([817542.5])

In [144]:
y_test


82015    640000.0
89400    265000.0
25683    720000.0
12291    620000.0
93745    358000.0
           ...   
4396     250000.0
93624    465000.0
11466    600000.0
11512    685000.0
48911    400888.0
Name: resale_price, Length: 31737, dtype: float64

In [142]:
X_sc = sc.transform([[148,1992,6,9,74,5000,-90000,2017]])
X_sc

array([[ 2.09527292, -0.22384279,  2.02984719, -0.08200504, -0.03113887,
         0.18981967, -1.80138779, -1.55238858]])

# Pipeline creation

In [114]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [123]:
pipeline = Pipeline([("scaler", StandardScaler()), ("model_rf", RandomForestRegressor())])

In [124]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model_rf', RandomForestRegressor())])

In [117]:
pipeline.score(X_test, y_test)

0.9382515631113202

In [135]:

new_file = open("rf.pkl", "wb")
pickle.dump(rf,new_file)

new_file.close()

In [118]:
from sklearn.metrics import r2_score

In [131]:
sc= StandardScaler()
X_tra_sc = sc.fit_transform(X_train)
X_tst_sc = sc.transform(X_test)


In [132]:
rf = RandomForestRegressor()
model_rf = rf.fit(X_tra_sc, y_train)

In [133]:
model_rf.score(X_tst_sc, y_test)

0.9383143373257777

In [ ]:
from sklearn.metrics import mean_squared_error
xgb_mse = mean_squared_error(y_test, y_pred_xgb, squared= False)
print("XGB accuracy", model_xgb.score(X_test_sc, y_test))
print("XGB RMSE", mean_squared_error(y_test, y_pred_xgb, squared = False))

In [ ]:
print("RandomForest accuracy", model_rf.score(X_test_sc, y_test))
print("Random Forest test performance", mean_squared_error(y_test, y_pred_rf, squared = False))

In [ ]:
sns.distplot(y_test-y_pred_rf)

In [ ]:
plt.scatter(y_test, y_pred_rf)

In [127]:
import pickle

file = open("randomforest_pipeline.pkl", "wb")
pickle.dump(pipeline,file)

file.close()

In [128]:
infile = open("randomforest_pipeline.pkl", "rb")
pick_model = pickle.load(infile)
infile.close()


In [129]:
pick_model.predict(X_test)

array([812865.        , 299347.85714286, 719821.90666667, ...,
       646960.        , 755026.66666667, 412790.83416958])

In [ ]:
pick_model


In [ ]:
pick_model.get_params().keys()

In [ ]:
X_train.head()

In [ ]:
y_train

In [ ]:
dt = X_train[0]
dt